## Horses or Humans using ImageDataGenerator

In [1]:
import tensorflow as tf
from tensorflow import keras

ACCRUACY_THRESHOLD = 0.999

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs['accuracy']) > ACCRUACY_THRESHOLD:
            print ("\nReached desired accuracy. Stops training")
            self.model.stop_training = True

In [2]:
import os
import zipfile

zipfilenameHorse     = 'C:/VIN_PYCHARM/ONE/Data/horses.zip'
zipfilenameHuman     = 'C:/VIN_PYCHARM/ONE/Data/humans.zip'
train_folder         = 'C:/VIN_PYCHARM/ONE/Data/temp/horse-or-human'
validation_folder    = 'C:/VIN_PYCHARM/ONE/Data/temp/validation-horse-or-human'

zipref = zipfile.ZipFile(zipfilenameHorse, 'r')
zipref.extractall(train_folder)
zipref = zipfile.ZipFile(zipfilenameHuman, 'r')
zipref.extractall(train_folder)

zipfilename = 'C:/VIN_PYCHARM/ONE/Data/validation-horse-or-human.zip'
zipref = zipfile.ZipFile(zipfilename, 'r')
zipref.extractall(validation_folder)

In [5]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)))
model.add(tf.keras.layers.MaxPool2D(2,2))

model.add(tf.keras.layers.Conv2D(32,(3,3), activation='relu'))
model.add(tf.keras.layers.MaxPool2D(2,2))

model.add(tf.keras.layers.Conv2D(64, (3,3),activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2,2))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [7]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(train_folder,
                                                    target_size =(150,150),
                                                    batch_size =128,
                                                    class_mode ='binary')

validattion_generator = validation_datagen.flow_from_directory(validation_folder,
                                                               target_size =(150,150),
                                                               batch_size =32,
                                                               class_mode ='binary')


Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [45]:
callbacks = myCallback()
history = model.fit(train_generator,
                    steps_per_epoch=8,
                    epochs=15,
                    verbose = 1,
                    validation_data=validattion_generator,
                    validation_steps=8,
                    callbacks=[callbacks])



Epoch 1/15
8/8 [==============================] - ETA: 0s - loss: 7.9536e-04 - accuracy: 1.0000
Reached desired accuracy. Stops training
8/8 [==============================] - 11s 1s/step - loss: 7.9536e-04 - accuracy: 1.0000 - val_loss: 2.0603 - val_accuracy: 0.8438


In [50]:
from keras_preprocessing import image


image_path = 'C:/VIN_PYCHARM/ONE/Data/test/human.png'
test_img_temp = image.load_img(image_path, target_size=(150,150))
test_img = image.img_to_array(test_img_temp)
test_img = test_img.reshape((1,) + test_img.shape)
test_img/= 255

classes = model.predict(test_img, batch_size=8)

print(classes[0])
if classes[0]>0.5:
    print("human")
else:
    print("horse")



[0.9999989]
human
